In [1]:
"""This is the sentiment analysis with Stopword, punctuation removal + POS tagging with adverb and adjective + emolex"""

'This is the sentiment analysis with Stopword, punctuation removal + POS tagging with adverb and adjective + emolex'

In [2]:
import os
import nltk
import random
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.text import Text
from nltk.classify import ClassifierI
import string, re
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from statistics import mode
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import mark_negation

documents = []
all_words = []
tokenizer = RegexpTokenizer(r'\w+')

stop_words = set(stopwords.words('english'))

positive_files_path = "./review_polarity/txt_sentoken/pos/"
negative_files_path = "./review_polarity/txt_sentoken/neg/"

allowed_pos = ['J', 'R']

for root, dirs, files in os.walk(positive_files_path, topdown=False):
    for file in files:
        filepath = os.path.join(root, file)
        text = open(filepath, 'r').read()
        words = tokenizer.tokenize(text)
        #mark_negation(words)
        documents.append((words, 'pos'))


        #tag and filter by allowed words
        tagged_words = nltk.pos_tag(words)
        for word in tagged_words:
            if word[0] not in stop_words and word[1][0] in allowed_pos:
                all_words.append(word[0])
        
for root, dirs, files in os.walk(negative_files_path, topdown=False):
    for file in files:
        filepath = os.path.join(root, file)
        text = open(filepath, 'r').read()
        words = tokenizer.tokenize(text)
        #mark_negation(words)
        documents.append((words, 'neg'))
        
        #tag and filter by allowed words
        tagged_words = nltk.pos_tag(words)
        for word in tagged_words:
            if word[0] not in stop_words and word[1][0] in allowed_pos:
                all_words.append(word[0])
                
emolex = open("./NRC_Emotion.txt", "r")
pos = {}
neg = {}
for line in emolex:
    words = line.strip()
    words = words.split('\t')
    if 'positive' in words and '1' in words:
        pos[words[0]] = bool(1)
    if 'negative' in words and '1' in words:
        neg[words[0]] = bool(1)
emolex.close

pos_featureset = [(pos, 'pos')]
neg_featureset = [(neg, 'neg')]

In [3]:
#function that finds features from a document
#features are top 20% most frequent words from all documents

all_words = nltk.FreqDist(all_words)
top_10 = int(len(all_words)*.20)
word_features, frequency = map(list, zip(*all_words.most_common(top_10)))

def find_features(document):
    words = set(document)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features

In [4]:
featuresets = [(find_features(review), category) for (review, category) in documents]
random.shuffle(featuresets)

training_set = featuresets[:1500] + pos_featureset + neg_featureset
testing_set = featuresets[1500:]
print(len(training_set))
print(len(testing_set))

1502
500


In [5]:
names = ['MultinomialNB','BernoulliNB', 
         'Decision_Tree',
         'Logistic Regression', 'Stochastic Gradient Descent',
         'SVC', 'LinearSVC', 'NuSVC']

trained = []

classifiers = [MultinomialNB(), BernoulliNB(), 
               DecisionTreeClassifier(random_state=0), 
               LogisticRegression(), SGDClassifier(),
               SVC(), LinearSVC(), NuSVC()]

for name, clf in zip(names, classifiers):
    print('Now classifying', name)
    
    classifier = SklearnClassifier(clf)
    classifier.train(training_set)
    trained.append(classifier)
    print("{} classifier accuracy:".format(name), nltk.classify.accuracy(classifier, testing_set))
    print("\n")

Now classifying MultinomialNB
MultinomialNB classifier accuracy: 0.858


Now classifying BernoulliNB
BernoulliNB classifier accuracy: 0.856


Now classifying Decision_Tree
Decision_Tree classifier accuracy: 0.602


Now classifying Logistic Regression
Logistic Regression classifier accuracy: 0.828


Now classifying Stochastic Gradient Descent
Stochastic Gradient Descent classifier accuracy: 0.822


Now classifying SVC
SVC classifier accuracy: 0.828


Now classifying LinearSVC
LinearSVC classifier accuracy: 0.804


Now classifying NuSVC
NuSVC classifier accuracy: 0.83




In [6]:
def analyze_text(trained, input_text):
    text = tokenizer.tokenize(input_text)
    features = find_features(text)
    return trained[3].classify(features)

In [7]:
#input the text you want sentiment analyzed in here:
#after inputting, please press shift + enter to run this cell.

sentiment_text = "This film was awesome, it took me back to when I was a child playing with my millennium falcon at home. It's fan service at its purest and repaired all the damage from the last film. Don't watch this film with a predetermined view of how you think this film should be, and watch it through a child's eyes as you did the first time you watched this franchise and just enjoy."

print("your input's sentiment is: ", analyze_text(trained, sentiment_text))

your input's sentiment is:  neg
